In [9]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [10]:
# Load data
df = pd.read_csv('ecommerce_customer_data_custom_ratios.csv')
df_clean = df.dropna()

In [11]:
# Remove outliers
def remove_outliers_iqr(data, column):
    Q1 = data[column].quantile(0.25)
    Q3 = data[column].quantile(0.75)
    IQR = Q3 - Q1
    lower = Q1 - 1.5 * IQR
    upper = Q3 + 1.5 * IQR
    return data[(data[column] >= lower) & (data[column] <= upper)]

numeric_cols = df_clean.select_dtypes(include=['float64', 'int64']).columns.drop(['Customer ID', 'Churn'])
df_outliers_removed = df_clean.copy()
for col in numeric_cols:
    df_outliers_removed = remove_outliers_iqr(df_outliers_removed, col)

In [12]:
# Preprocess
df_model = df_outliers_removed.drop(columns=['Customer ID', 'Purchase Date', 'Customer Name'])
le = LabelEncoder()
for col in df_model.select_dtypes(include='object').columns:
    df_model[col] = le.fit_transform(df_model[col])


In [13]:
# Stratified sample
df_sampled = df_model.groupby('Churn', group_keys=False).apply(lambda x: x.sample(frac=0.1, random_state=42))


C:\Users\User\AppData\Local\Temp\ipykernel_14460\3456566000.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_sampled = df_model.groupby('Churn', group_keys=False).apply(lambda x: x.sample(frac=0.1, random_state=42))


In [14]:
# Split
X = df_sampled.drop(columns='Churn')
y = df_sampled['Churn']
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.2, random_state=42)

In [15]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

rf = RandomForestClassifier(random_state=42)
rf.fit(X_train, y_train)
rf_preds = rf.predict(X_test)

print("Random Forest Accuracy:", accuracy_score(y_test, rf_preds))
print("Random Forest Report:\n", classification_report(y_test, rf_preds))


Random Forest Accuracy: 0.7989130434782609
Random Forest Report:
               precision    recall  f1-score   support

           0       0.80      0.99      0.89      3241
           1       0.37      0.01      0.02       807

    accuracy                           0.80      4048
   macro avg       0.59      0.50      0.46      4048
weighted avg       0.72      0.80      0.72      4048



In [16]:
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier()
knn.fit(X_train, y_train)
knn_preds = knn.predict(X_test)

print("KNN Accuracy:", accuracy_score(y_test, knn_preds))
print("KNN Report:\n", classification_report(y_test, knn_preds))



KNN Accuracy: 0.7677865612648221
KNN Report:
               precision    recall  f1-score   support

           0       0.80      0.95      0.87      3241
           1       0.19      0.05      0.08       807

    accuracy                           0.77      4048
   macro avg       0.50      0.50      0.47      4048
weighted avg       0.68      0.77      0.71      4048

